# Chatbot Practice

## Dependencies

In [20]:
!pip install --quiet ipywidgets openai

In [19]:
import os
import openai 
from langchain.chat_models import ChatOpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output
from dotenv import load_dotenv
# Load environment variables (you'll need to create a .env file with your OpenAI API key)
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

## Basic Chatbot

In [43]:
client = openai.OpenAI()


In [44]:
def chat_with_model(messages, model="gpt-4.1-nano-2025-04-14"):
    """
    Send a list of messages to the OpenAI ChatCompletion endpoint.
    messages: List of dicts, e.g. [{"role": "user", "content": "Hello!"}]
    Returns the assistant's reply as a string.
    """
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content

In [45]:
def console_chat():
    """
    A basic console-based chat loop.
    """
    print("Welcome to the OpenAI console chatbot! Type 'exit' to quit.")
    messages = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ("exit", "quit"):
            print("Goodbye!")
            break
        messages.append({"role": "user", "content": user_input})
        reply = chat_with_model(messages)
        messages.append({"role": "assistant", "content": reply})
        print(f"Bot: {reply}\n")


### UI Screen

In [ ]:
# Prepare state
messages = []

# Widgets: input box, send button, output area
input_box = widgets.Text(
    placeholder='Type your message...',
    description='You:',
    layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(description='Send')
output_area = widgets.Output(layout={'border': '1px solid lightgray'})

# Send callback
def on_send(b):
    with output_area:
        clear_output(wait=True)  # Clear previous output
        user_text = input_box.value.strip()
        if not user_text:
            return
        
        # Add user message to conversation
        messages.append({"role": "user", "content": user_text})
        
        # Display conversation history
        for msg in messages:
            role = "You" if msg["role"] == "user" else "Bot"
            print(f"{role}: {msg['content']}")
        
        # Get bot response
        bot_reply = chat_with_model(messages)
        messages.append({"role": "assistant", "content": bot_reply})
        
        # Display bot's response
        print(f"Bot: {bot_reply}")
        
        input_box.value = ''

send_button.on_click(on_send)

# Display UI
display(input_box, send_button, output_area)


Print the current message history


In [ ]:
print("Current message history:")
for i, message in enumerate(messages):
    role = message["role"].capitalize()
    content = message["content"]
    print(f"{i+1}. {role}: {content}")
    print("-" * 50)

if not messages:
    print("No messages in history yet.")


## With System Prompt

Reminder to system prompts do's and dont's:

Do's:
* *Role or persona*
e.g., “You are a professional tech support agent…”
* *Tone or style*
e.g., “Respond in a friendly and concise manner.”
* *Language / format instructions*
e.g., “Use markdown for code”, “Respond in English”
* *Constraints / safety rules*
e.g., “Never answer medical questions”, “Say 'I don’t know' if unsure”

Dont's:
* Overly vague instructions ("Be helpful")
* Tasks better handled dynamically in user prompt
* Large documents or context (use RAG instead)
* Long prompt chains — system prompts should be short & stable



In [32]:
# Define a system prompt to guide the assistant's behavior
system_prompt = {
    "role": "system",
    "content": (\
        """
        You are a virtual pizza ordering assistant for a restaurant called "Pizza Planet".

Your job is to take pizza orders from users in a friendly, professional way and summarize the full order with a total price.

You must follow these rules:

1. Always use a fixed toppings menu with prices:
   - Mushrooms ($2)
   - Olives ($1.5)
   - Onions ($1)
   - Extra cheese ($2)
   - Pepperoni ($2.5)
   - Pineapple ($1.5)

2. Ask for:
   - Pizza size (Small - $8, Medium - $10, Large - $12)
   - Toppings (from list above)
   - Pickup or delivery (Delivery costs $3 extra)

3. At the end:
   - Summarize the full order
   - Calculate and state the total price
   - Offer a **drink** (soda $2) or **dessert** (chocolate lava cake $3) as an optional upsell

4. If the user asks for toppings not on the list, politely say they're not available.

Use short, clear messages. Always confirm the full order before placing it.

       
        """
    )
}



In [ ]:
system_prompt = {
    "role": "system",
    "content": (
        "אתה מוקד חירום רשמי בישראל. תפקידך לעזור למתקשרים ולנתב אותם לשירות החירום המתאים.\n\n"
        "שירותי החירום הזמינים:\n"
        "- משטרה (100): עבירות, תאונות דרכים, אלימות, גניבות\n"
        "- מגן דוד אדום (101): פציעות, מחלות, לידה, מצבי חירום רפואיים\n"
        "- כיבוי אש (102): שריפות, תאונות עם כלי רכב, הצפות, חילוץ\n\n"
        "הוראות:\n"
        "1. ענה בצורה מקצועית ורגועה\n"
        "2. אסוף מידע חיוני: מיקום, תיאור האירוע, מספר נפגעים\n"
        "3. אם יש סכנת חיים מיידית, הדגש את חשיבות הקריאה המיידית\n"
        "4. תן הנחיות בטיחות בסיסיות אם רלוונטי\n"
        "5. אל תספק מידע רפואי מקצועי - רק הנחיות בסיסיות"
    )
}

In [49]:
def chat_with_model(messages, model="gpt-4.1-nano-2025-04-14"):
    """
    Send messages to the v1 API, prepending the system prompt, and return the assistant's reply.
    """
    full_messages = [system_prompt] + messages
    response = client.chat.completions.create(
        model=model,
        messages=full_messages
    )
    return response.choices[0].message.content

In [ ]:
# Conversation history (user/assistant only)
messages = []

# Widgets: input, button, output
input_box = widgets.Text(
    placeholder='Type here...', description='You:', layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(description='Send')
output_area = widgets.Output(layout={'border': '1px solid lightgray'})

# On send handler
def on_send(b):
    with output_area:
        clear_output()
        user_text = input_box.value.strip()
        if not user_text:
            return
        print(f"You: {user_text}")
        messages.append({"role": "user", "content": user_text})
        bot_reply = chat_with_model(messages)
        messages.append({"role": "assistant", "content": bot_reply})
        print(f"Bot: {bot_reply}")
        input_box.value = ''

send_button.on_click(on_send)

# Display the chat UI
display(input_box, send_button, output_area)

In [ ]:
print("Current message history:")
for i, message in enumerate(messages):
    role = message["role"].capitalize()
    content = message["content"]
    print(f"{i+1}. {role}: {content}")
    print("-" * 50)

if not messages:
    print("No messages in history yet.")